CNN version 4.0 This one picks up from previous version and moves further
=============
Using the preprocessed Data
------------

Building upon the previous cnn version. 

This notebook uses a more sophisticated and a deeper cnn than the previous.
Use of TensorBoard for visualization

In [12]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from Helpers.CNN_Creator import *

# Task 1: Load the data from the pickle file

In [2]:
'''Set the constant paths here'''
# set the paths required for the script to work:
root = "../Data/"
pickle_file_path = root + "Data_pindown.pickle"
log_path = root + "logs"
model_path = os.path.join(root, "Models/Model1/")

Retrieve the saved data from the pickle file

In [3]:
Formatted_Data = {}
with open(pickle_file_path, "rb") as f:
    Formatted_Data = pickle.load(f)

In [4]:
len(Formatted_Data)

7

In [5]:
# recollect the fragmented data

labels_mappings = Formatted_Data['label_mapping']
print(labels_mappings)

train_dataSet = Formatted_Data['train_data']
train_labels  = Formatted_Data['train_labels']

cv_dataSet = Formatted_Data['cv_data']
cv_labels = Formatted_Data['cv_labels']

test_dataSet = Formatted_Data['test_data']
test_labels  = Formatted_Data['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'Action-Painting': 0, 'Colour-Field-Painting': 1, 'Surreal-Surrealism': 8, 'art--Expressionism': 9, 'Op-Art': 6, 'Hard-Edge-Art': 3, 'ORPHISM': 5, 'Post-impressionism': 7, 'Minimalism-Art': 4, 'Cubism': 2}
(1927, 100, 100, 3) (276, 100, 100, 3) (551, 100, 100, 3)
(1927, 10) (276, 10) (551, 10)


In [6]:
full_dataSet = np.vstack((train_dataSet, cv_dataSet, test_dataSet))
full_labels = np.vstack((train_labels, cv_labels, test_labels))

# Task 2: Spawn the CNN 

In [7]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, axis=1) == np.argmax(labels, axis=1)) / predictions.shape[0])

In [33]:
# define the constants required hence forth:
batch_size = 900
patch_size = 5
num_hidden = 256

image_size = train_dataSet.shape[1]
num_labels = len(labels_mappings)
num_channels = 3 # rgb images

### The CNN tensorflow graph 

In [34]:
graph = tf.Graph()

''' Neural network name: BotNet
    Size: medium
    conf: conv => conv => pool => conv => pool => fc => fc => softmax'''

with graph.as_default():
    
    # Input data.
    with tf.name_scope("INPUTS"):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name="Data")
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="Labels")

        tf_valid_dataset = tf.constant(cv_dataSet[:100], name="cross_validation_dataset")
        tf_full_dataset = tf.constant(test_dataSet, name="test_dataset")
        
    # Model.
    def model(data):
        # CONV LAYER
        # layer has 8 (3 x 3) convoluting filters
        layer1_1 = addConvLayer(data, [patch_size, patch_size, num_channels, 8], 
                                "lay_1.1_w", "lay_1.1_b", tf.nn.sigmoid)
    
        
        # CONV LAYER  
        # layer has 12 (3 x 3) convoluting filters
        layer2_1 = addConvLayer(layer1_1, [patch_size, patch_size, 8, 12], 
                                "lay_2.1_w", "lay_2.1_b", tf.nn.sigmoid)
    
    
        # CONV LAYER
        # layer has 16 (3 x 3) convoluting filters
        layer3_1 = addConvLayer(layer2_1, [patch_size, patch_size, 12, 16], 
                                "lay_3.1_w", "lay_3.1_b", tf.nn.sigmoid)
    
        
        # POOL LAYER
        # will not require any data variable or parameter holder
        # a simple 2 x 2 max_pooling operation
        layer4_1 = addPoolLayer(layer3_1, [1, 2, 2, 1], [1, 2, 2, 1])
        
        # CONV LAYER
        # layer has 20 (3 x 3) convoluting filters
        layer5_1 = addConvLayer(layer4_1, [patch_size, patch_size, 16, 20],
                                "lay_5.1_w", "lay_5.1_b", tf.nn.sigmoid)
        
        # FC LAYER
        # input dimension will be: 8 * 8 * 32
        # length, width calculation: 32 =first pool=> 16 =second pool=> 8
        # depth calculation: 32 ...(same as the last conv output)
        # output dimension: 128 ...(equal to number of hidden cells)
        # number of hidden cells: 128
        # reshape the tensor before passing it to the FC layers
        shape = layer5_1.get_shape().as_list()
        reshape = tf.reshape(layer5_1, [shape[0], shape[1] * shape[2] * shape[3]])
        layer6_1 = addFCLayer(reshape, [shape[1] * shape[2] * shape[3], num_hidden],
                              "lay_6.1_w", "lay_6.1_b", tf.nn.sigmoid)
        
        # FC LAYER
        # input dimension will be: 256
        # output dimension will be: 256
        # number of hidden cells: 256
        layer7_1 = addFCLayer(layer6_1, [num_hidden, num_hidden],
                              "lay_7.1_w", "lay_7.1_b", tf.nn.sigmoid)
        
        # Fianl output layer for the neural network
        layer8_1 = addFCLayer(layer7_1, [num_hidden, num_labels],
                              "lay_8.1_w", "lay_8.1_b", tf.nn.sigmoid)
        
        return layer8_1
  
    # Training computation.
    logits = model(tf_train_dataset)
    
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        tf.summary.scalar("loss", loss)
        
    # Optimizer.
    with tf.name_scope("train_step"):
        optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    with tf.name_scope("train_prediction"):
        train_prediction = tf.nn.softmax(logits)
    
    with tf.name_scope("validation_prediction"):
        test_prediction = tf.nn.softmax(model(tf_full_dataset))

### Graph Excuter Session module

In [37]:
num_steps = 1001
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
with tf.Session(graph=graph) as session:
    saver = tf.train.Saver()
    
    train_writer = tf.summary.FileWriter(os.path.join(log_path, "train"), session.graph)
    merged = tf.summary.merge_all()
    
    # restore the saved model from the path
    saver.restore(session, os.path.join(model_path, "mod_1.ckpt"))
    
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = full_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = full_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 200 == 0):
            summary = session.run(merged, feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            print(predictions[:3])
            print(batch_labels[:3])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
    
    # rewrite the previously saved model
    saver.save(session, os.path.join(model_path, "mod_1.ckpt"))

Initialized
[[ 0.07529878  0.10404339  0.07514746  0.07512595  0.19676998  0.07511509
   0.17197362  0.07533728  0.07607376  0.07511467]
 [ 0.12470678  0.09548815  0.1017364   0.09610452  0.09541789  0.10092862
   0.09550294  0.09919193  0.0954067   0.09551623]
 [ 0.08167028  0.08274936  0.11171994  0.08149166  0.08151009  0.08149213
   0.08149834  0.21919896  0.09692203  0.08174723]]
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]
Minibatch loss at step 0: 1.612513
Minibatch accuracy: 82.2%
[[ 0.21128124  0.07895117  0.07896396  0.08619419  0.0949732   0.07894557
   0.1338779   0.07893623  0.07893644  0.07894015]
 [ 0.08697365  0.09242021  0.07541528  0.07541883  0.20310141  0.07541399
   0.16373631  0.07560114  0.07650521  0.07541396]
 [ 0.08991421  0.09430077  0.08445638  0.21983472  0.08432731  0.08959834
   0.08447454  0.08432452  0.08432221  0.08444689]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [